In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import csv
import nltk
import re
from nltk.stem import WordNetLemmatizer
import sklearn
import codecs
import pymorphy2
import seaborn as sns
sns.set_style("darkgrid")
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from nltk.corpus import stopwords

In [2]:
class website:
    def __init__(self, CSV_POS='', CSV_NEG='', text_pos={}, text_neg={},
                 tf_idf_matrix_pos='', tf_idf_matrix_neg='', web_name='',
                 words_pos=[], words_neg=[]):
        self.csv = {
            "pos": CSV_POS,
            "neg": CSV_NEG
        }
        self.text = {
            "pos": text_pos,
            "neg": text_neg
        }
        self.tfidf_matrix = {
            "pos":  tf_idf_matrix_pos,
            "neg": tf_idf_matrix_neg
        }
        self.name = web_name
        self.words = {
            "pos": words_pos,
            "neg": words_neg
        }

In [3]:
def list_to_str(arr):
    s = ''
    for rec in arr:
        s += rec
    return df_preprocess_revs(s)

In [4]:
def csv_to_list(arr):
    arr_list = []
    for row in arr:
        arr_list.append(list_to_str(row))
    return arr_list

In [5]:
def key(x):
    try:
        return countt[x]
    except KeyError:
        return 0

In [6]:
def clasterization(w, st):
    print(f'\t{st} reviews from {w.name}: clasterization...')
    num_clusters = 10
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    kmeans.fit(w.tfidf_matrix[st])
    for cluster_id in range(num_clusters):
        cluster_indices = np.where(kmeans.labels_ == cluster_id)[0]
        print(f'\nКластер {cluster_id + 1}: (кол-во отзывов: {len(cluster_indices)}) ------------\n')
        ch = 0
        for idx in sorted(cluster_indices, key=lambda x: countt[w.words[st][x]], reverse=True):
            if ch == 20:
                break
            if (rev_list[idx] != ''):
                print(f"{rev_list[idx][:-12:]}: {countt[w.words[st][idx]]}")
            ch += 1
    print("--------")

In [7]:
def vectorization(w, st):
    print(f'\t{st} reviews from {w.name}: vectorization...')
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(w.text[st]['Text'])
    count_reviews = vectors.shape[0]
    count_tockens = vectors.shape[1]

    tf_idf = pd.DataFrame(vectors.todense()).iloc[:count_reviews]
    tf_idf.columns = vectorizer.get_feature_names_out()
    w.words[st] = tf_idf.columns    # сохранение для кластеризации
    tfidf_matrix = tf_idf.T
    tfidf_matrix.columns = ['response' + str(i) for i in range(1, count_reviews + 1)]
    tfidf_matrix['count'] = tfidf_matrix.sum(axis=1)
    for i in range(len(tfidf_matrix['count'])):
        countt[w.words[st][i]] = tfidf_matrix['count'][i]

    tfidf_matrix = tfidf_matrix.sort_values(by ='count', ascending=False)
    print(tfidf_matrix.drop(columns=['count']).head(count_tockens))
    return tfidf_matrix.T

In [8]:
def df_preprocess(text):
    reg = re.compile(r'[^а-яА-яa-zA-Z0-9 ]')
    text = text.lower().replace("ё", "е")
    text = text.replace("ъ", "ь")
    text = text.replace("й", "и")
    text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', 'сайт', text)
    text = re.sub(r'@[^\s]+', 'пользователь', text)
    text = reg.sub(' ', text)
    stopWords = set(stopwords.words('russian'))
    # Стемминг
    stemmer = SnowballStemmer("russian")
    text = ' '.join([stemmer.stem(word) for word in text.split() if word not in stopWords])
    return text

In [9]:
def df_preprocess_revs(text):
    reg = re.compile(r'[^а-яА-яa-zA-Z0-9 ]')
    text = reg.sub(' ', text)
    return text

In [10]:
statuses = ["neg", "pos"]

In [11]:
# websites init
rustore = website()
rustore.name = 'rustore.ru'
for st in statuses:
    rustore.csv[st] = fr'result\api\rustore.ru\csv\revData_rustore.ru_{st}.csv'
    rustore.text[st] = pd.read_csv(rustore.csv[st])
    rustore.text[st]['Text'] = rustore.text[st]['Text'].apply(df_preprocess)

FileNotFoundError: [Errno 2] No such file or directory: 'result\\api\\rustore.ru\\csv\\revData_rustore.ru_neg.csv'

In [ ]:
sravni = website()
sravni.name = 'sravni.ru'
for st in statuses:
    sravni.csv[st] = fr'result\api\sravni.ru\csv\revData_sravni.ru_{st}.csv'
    sravni.text[st] = pd.read_csv(sravni.csv[st])
    sravni.text[st]['Text'] = sravni.text[st]['Text'].apply(df_preprocess)

In [ ]:
for st in statuses:
    countt = {}
    rustore.tfidf_matrix[st] = vectorization(rustore, st)
    with open(rustore.csv[st], 'r', encoding='utf-8') as file:
        q = csv.reader(file)
        rev_list = csv_to_list(q)
    clasterization(rustore, st)
    sravni.tfidf_matrix[st] = vectorization(sravni, st)
    with open(sravni.csv[st], 'r', encoding='utf-8') as file:
        q = csv.reader(file)
        rev_list = csv_to_list(q)
    clasterization(sravni, st)